In [ ]:
import pandas as pd



In [ ]:
from google.colab import files
uploaded = files.upload()



Saving sarcasm_detection_dataset.csv to sarcasm_detection_dataset.csv


In [ ]:
data = pd.read_csv("sarcasm_detection_dataset.csv")

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, concatenate, Dropout
from sklearn.preprocessing import MinMaxScaler

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['comment'] + data['parent_comment'])
comment_seq = tokenizer.texts_to_sequences(data['comment'])
parent_comment_seq = tokenizer.texts_to_sequences(data['parent_comment'])

comment_padded = pad_sequences(comment_seq, maxlen=100, padding='post', truncating='post')
parent_comment_padded = pad_sequences(parent_comment_seq, maxlen=100, padding='post', truncating='post')

# Combine the two sequences
combined_text_data = np.hstack([comment_padded, parent_comment_padded])


In [ ]:
scaler = MinMaxScaler()
numerical_data = data[['child_is_neutral', 'child_is_positive', 'child_is_negative',
                       'child_neg_score', 'child_neu_score', 'child_pos_score',
                       'parent_is_negative', 'parent_is_neutral', 'parent_is_positive',
                       'parent_neg_score', 'parent_neu_score', 'parent_pos_score']]

numerical_data_normalized = scaler.fit_transform(numerical_data)

In [ ]:
text_input = Input(shape=(200,))
text_model = Embedding(input_dim=5000, output_dim=16)(text_input)
text_model = LSTM(32, return_sequences=True)(text_model)
text_model = LSTM(32)(text_model)

In [ ]:
numerical_input = Input(shape=(12,))
numerical_model = Dense(32, activation='relu')(numerical_input)

In [ ]:
combined = concatenate([text_model, numerical_model])
combined = Dense(32, activation='relu')(combined)
combined = Dropout(0.5)(combined)
output = Dense(1, activation='sigmoid')(combined)

model = Model(inputs=[text_input, numerical_input], outputs=output)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit([combined_text_data, numerical_data_normalized], data['label'], epochs=10)

loss, accuracy = model.evaluate([combined_text_data, numerical_data_normalized], data['label'])
print(f"Model accuracy: {accuracy * 100:.2f}%")

Epoch 1/10
3125/3125 [==============================] - 109s 32ms/step - loss: 0.6927 - accuracy: 0.5128
Epoch 2/10
3125/3125 [==============================] - 53s 17ms/step - loss: 0.6912 - accuracy: 0.5238
Epoch 3/10
3125/3125 [==============================] - 51s 16ms/step - loss: 0.6887 - accuracy: 0.5320
Epoch 4/10
3125/3125 [==============================] - 50s 16ms/step - loss: 0.6848 - accuracy: 0.5369
Epoch 5/10
3125/3125 [==============================] - 49s 16ms/step - loss: 0.6792 - accuracy: 0.5416
Epoch 6/10
3125/3125 [==============================] - 50s 16ms/step - loss: 0.6742 - accuracy: 0.5460
Epoch 7/10
3125/3125 [==============================] - 50s 16ms/step - loss: 0.6702 - accuracy: 0.5470
Epoch 8/10
3125/3125 [==============================] - 49s 16ms/step - loss: 0.6683 - accuracy: 0.5509
Epoch 9/10
3125/3125 [==============================] - 48s 15ms/step - loss: 0.6668 - accuracy: 0.5505
Epoch 10/10
3125/3125 [==============================] - 24s 8m